In [ ]:
# -*- coding: utf-8 -*-
import io
import unicodedata
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st

# ---------------------------
# 1. RECUPERACIÓN DE CONTEXTO (APP.PY)
# ---------------------------
# Recuperamos variables inyectadas por app.py
estado_seleccionado = locals().get("ESTADO_SELECCIONADO", "Ciudad de México")
palette = locals().get("active_palette", ["#0576F3", "#36F48C", "#F47806"])
active_font = locals().get("active_font", "Sans-serif")

# Asignación de colores basada en la paleta de la app
# Usamos el ciclo de la paleta para asegurar que siempre haya color
def get_color(idx):
    return palette[idx % len(palette)]

COLOR_TOTAL   = get_color(0)
COLOR_HOMBRES = get_color(1)
COLOR_MUJERES = get_color(2)

CSV_URL = (
    "https://www.datos.gob.mx/dataset/"
    "f2b9b220-3ef7-4e3a-bde6-87e1dac78c6a/resource/"
    "3c3092be-583e-4490-8c23-67ef9a64b198/download/pobproy_quinq1.csv"
)

TEMPLATE = "plotly_white"

# ---------------------------
# 2. FUNCIONES DE PROCESAMIENTO
# ---------------------------
def _strip_accents(s: str) -> str:
    s = str(s) if s is not None else ""
    s_norm = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s_norm if not unicodedata.combining(ch)).lower().strip()

def _fmt_es(n) -> str:
    """Formatea miles con punto."""
    try:
        return f"{int(round(float(n))):,}"
    except Exception:
        return str(n)

@st.cache_data(show_spinner=False)
def descargar_csv(url: str) -> pd.DataFrame:
    """Descarga el CSV con caché de Streamlit para no repetir la descarga."""
    try:
        return pd.read_csv(url, low_memory=False)
    except UnicodeDecodeError:
        return pd.read_csv(url, encoding="latin-1", low_memory=False)

def normalizar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    cmap = {c.lower(): c for c in df.columns}
    def pick(*cands):
        for c in cands:
            if c.lower() in cmap:
                return cmap[c.lower()]
        for c in cands:
            for col in cmap:
                if c.lower().replace("ñ","n") == col.replace("ñ","n"):
                    return cmap[col]
        return None # Retorna None si no encuentra

    # Intentos de mapeo
    col_ent  = pick("NOM_ENT","Entidad","nombre_entidad")
    col_mun  = pick("NOM_MUN","municipio","nombre_municipio")
    col_sexo = pick("SEXO","sexo")
    col_anio = pick("ANO","AÑO","anio","year")
    col_pob  = pick("POB_TOTAL","poblacion","valor")

    if not all([col_ent, col_sexo, col_anio, col_pob]):
        raise KeyError("No se encontraron todas las columnas necesarias en el CSV de CONAPO.")

    out = df.rename(columns={col_ent:"NOM_ENT", col_mun:"NOM_MUN", col_sexo:"SEXO", col_anio:"ANO", col_pob:"POB_TOTAL"})
    # Filtramos solo columnas útiles
    cols_utiles = [c for c in ["NOM_ENT","NOM_MUN","SEXO","ANO","POB_TOTAL"] if c in out.columns]
    return out[cols_utiles]

def agregar_por_entidad(df: pd.DataFrame, estado: str) -> pd.DataFrame:
    estado_key = _strip_accents(estado)
    sel = df[df["NOM_ENT"].apply(lambda s: _strip_accents(s) == estado_key)].copy()

    if sel.empty:
        # Fallback simple si no encuentra
        raise ValueError(f"No se encontraron datos para el estado: {estado}")

    sel["ANO"] = pd.to_numeric(sel["ANO"], errors="coerce").astype("Int64")
    sel["POB_TOTAL"] = pd.to_numeric(sel["POB_TOTAL"], errors="coerce")

    grp = sel.groupby(["ANO","SEXO"], as_index=False)["POB_TOTAL"].sum()
    piv = grp.pivot_table(index="ANO", columns="SEXO", values="POB_TOTAL", aggfunc="sum").fillna(0)

    piv = piv.rename(columns={c: c.title() for c in piv.columns})

    # Calcular Total si no existe
    if "Total" not in piv.columns:
        cols_sum = [c for c in ["Hombres", "Mujeres"] if c in piv.columns]
        if cols_sum:
            piv["Total"] = piv[cols_sum].sum(axis=1)
        else:
            piv["Total"] = piv.sum(axis=1)

    return piv

def recortar_2025_2030(piv: pd.DataFrame) -> pd.DataFrame:
    target_years = list(range(2025, 2031))
    cols = [c for c in ["Total","Hombres","Mujeres"] if c in piv.columns]
    # Intersección de años disponibles y target
    years_avail = [y for y in target_years if y in piv.index]

    if not years_avail:
          raise ValueError("El rango de años 2025-2030 no está disponible en los datos.")

    out = piv.loc[years_avail, cols].astype("int64")
    return out

def _preparar_largo(tabla: pd.DataFrame) -> pd.DataFrame:
    largo = tabla.reset_index().melt(id_vars="ANO", var_name="Categoría", value_name="Población")
    largo["Año"] = largo["ANO"].astype(int).astype(str)
    largo["Etiqueta"] = largo["Población"].apply(_fmt_es)
    return largo

def graficar_streamlit(largo: pd.DataFrame, estado: str):
    # Mapa de colores dinámico
    color_map = {"Total": COLOR_TOTAL, "Hombres": COLOR_HOMBRES, "Mujeres": COLOR_MUJERES}

    # Asegurar orden
    cats_presentes = [c for c in ["Total", "Hombres", "Mujeres"] if c in largo["Categoría"].unique()]

    fig = px.bar(
        largo,
        x="Año",
        y="Población",
        color="Categoría",
        color_discrete_map=color_map,
        barmode="group",
        text="Etiqueta",
        category_orders={"Categoría": cats_presentes},
        template=TEMPLATE
    )

    fig.update_layout(
        title={
            "text": f"Proyección CONAPO: {estado} (2025 - 2030)",
            # CAMBIO 1: Título centrado
            "x": 0.5,
            "xanchor": "center"
        },
        font=dict(family=active_font, size=14),
        # CAMBIO 2: Leyenda inferior centrada
        legend=dict(
            orientation="h",
            yanchor="top", y=-0.2, # Posición Y negativa para moverla abajo
            xanchor="center", x=0.5
        ),
        # CAMBIO 3: Aumento de margen inferior para dar espacio a la leyenda
        margin=dict(l=20, r=20, t=60, b=100), 
        separators=".,"
    )

    fig.update_traces(
        textposition="outside",
        texttemplate="%{text}",
        hovertemplate="<b>%{x}</b><br>%{legendgroup}: %{customdata} hab.<extra></extra>",
        customdata=np.array(largo["Etiqueta"])
    )

    fig.update_yaxes(showgrid=True, gridcolor="#eee", zeroline=False, tickformat=",d")
    fig.update_xaxes(title=None)

    st.plotly_chart(fig, use_container_width=True)

# ---------------------------
# 3. EJECUCIÓN PRINCIPAL
# ---------------------------

if not estado_seleccionado:
    st.warning("⚠️ No hay un estado seleccionado.")
else:
    # Spinner de Streamlit para feedback visual durante la descarga
    with st.spinner(f"📥 Descargando proyecciones de CONAPO para {estado_seleccionado}..."):
        try:
            # 1. Descarga
            df_raw = descargar_csv(CSV_URL)

            # 2. Proceso
            df = normalizar_columnas(df_raw)
            piv = agregar_por_entidad(df, estado_seleccionado)
            tabla = recortar_2025_2030(piv)
            largo = _preparar_largo(tabla)

            # 3. Gráfica
            graficar_streamlit(largo, estado_seleccionado)

            # Nota informativa (Fuente consultada, queda debajo de la gráfica)
            st.caption("Fuente: Proyecciones de la Población de México y de las Entidades Federativas, CONAPO (Datos Abiertos).")

        except Exception as e:
            st.error(f"❌ Ocurrió un error al procesar los datos: {e}")